In [67]:
import pandas as pd
import numpy as np
import os
from sklearn.impute import KNNImputer
import shap
import sklearn
from sklearn.metrics import matthews_corrcoef
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import classification_report, roc_auc_score,accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import re
from sklearn.model_selection import cross_val_score,KFold, LeaveOneOut
from matplotlib import pyplot
from sklearn.ensemble import VotingClassifier
from sklearn.preprocessing import MinMaxScaler, minmax_scale
from sklearn.feature_selection import SelectKBest, chi2, RFE, SelectFromModel
import xgboost as xgb
from sklearn.preprocessing import StandardScaler
pd.set_option('display.max_rows', None)

In [68]:
filename = ''
output_directory = ''
df = pd.read_excel(filename)
df.set_index('Pat', inplace=True)

In [69]:
#Removing all unnecessary variables
columns_drop = ['SAL', 'AGEBIN', 'Sex male', 'sex female', 'EXAML', 'SEX',
                'AMLSTAT', 'WBCU', 'HB', 'HBU', 'PLT', 'PLTU', 'LDH', 'LDHU', 'CGKT',
                'ELN2017', 'ELN2022', 'CEBPASTAT', 'FEV',
               'ELN2022 fav', 'ELN2022 int', 'ELN2022 adv', 'ALSCTCR1', 'ALSCTSLV', 'ALSCTOTH',
                'OSTMU', 'EFSTM','CGNK', 'CGCX',
                'EFSTMU', 'EFSSTAT', 'RFSTM', 'FLT3R',
               'RFSTMU', 'RFSSTAT', 'Unnamed: 0', 'CEBPADM', 'CEBPA.bZIP','CEBPA.bZIP.inframe', 'CEBPA.TAD',
               'OSTM_unter24_zensiert', 'OSTM = 24', 'OSTMabove24', 'OSSTAT', 'OSTM','FLT3T', 'TrialID',
                'WBC', 'BMB', 'PBB'] 
df = df.drop(columns_drop, axis=1)

In [70]:
replacement_mappings = {
    "t8.21": "t(8;21)",
    "minus.5": "-5",
    "minus.7": "-7",
    "minus.17": "-17",
    "del.5q.": "del(5q)",
    "t.v.11..v.q23.": "t(v;11q23.3)",
    "t.9.11..p21.23.q23.": "t(9;11)(p21.3;q23.3)",
    "t.10.11.": "t(10;11)",
    "t.11.19..q23.p13.": "t(11;19)(q23.3;p13.3)",
    "del.7q.": "del(7q)",
    "del.9q.": "del(9q)",
    "minus.X": "-X",
    "minus.Y": "-Y",
    "inv16_t16.16": "inv(16)(p13.1q22)",
    "CEBPA.bZIP.inframe": "CEBPA-bZip-inf",
    "FLT3I": "FLT3-ITD"
}

# Loop through each column where replacements are needed
for column in df.columns:
    df.rename(columns=replacement_mappings, inplace=True)


In [71]:
#Removing all mutations which have less than 1% of occurence
sum=0
columns = df.loc[:, (df.columns != 'AGE') & (df.columns != 'OSTM') & (df.columns != 'TrialID') & (df.columns != 'AGEGRP')
                & (df.columns != 'WBC') & (df.columns != 'BMB') & (df.columns != 'PBB')]

for column in columns:
    counter = (df[column].values == 1).sum()
    patients = len(df)
    result=counter/patients
    if (counter/patients) <= 0.01:
        sum+=1
        print(str(column) + ': ' + str(counter) + '/' + str(patients) + ' = ' + str(result))
        df = df.drop(column, axis=1)

ATRX: 7/3062 = 0.0022860875244937948
BRAF: 12/3062 = 0.003919007184846506
CALR: 8/3062 = 0.002612671456564337
CBLB: 4/3062 = 0.0013063357282821686
CDKN2A: 6/3062 = 0.001959503592423253
FBXW7: 4/3062 = 0.0013063357282821686
GATA1: 8/3062 = 0.002612671456564337
GNAS: 4/3062 = 0.0013063357282821686
HRAS: 2/3062 = 0.0006531678641410843
KDM6A: 23/3062 = 0.007511430437622469
MPL: 10/3062 = 0.0032658393207054214
MYD88: 3/3062 = 0.0009797517962116264
PDGFRA: 2/3062 = 0.0006531678641410843
PTEN: 6/3062 = 0.001959503592423253
SETBP1: 17/3062 = 0.005551926845199217
t.6.9..p23.q34.: 17/3062 = 0.005551926845199217
inv.3..q21.q26.2.: 20/3062 = 0.006531678641410843
t.9.22..q34.q11.: 2/3062 = 0.0006531678641410843
abn.17p.: 29/3062 = 0.009470934030045721
t.3.5.: 10/3062 = 0.0032658393207054214
t.6.11.: 19/3062 = 0.0062050947093403004
t(11;19)(q23.3;p13.3): 27/3062 = 0.008817766165904637


In [72]:
df["inv(16)(p13.1q22)"] = pd.to_numeric(df["inv(16)(p13.1q22)"], errors='coerce')
df["t(8;21)"] = pd.to_numeric(df["t(8;21)"], errors='coerce')
print(df.shape)

for index, row in df.iterrows():
    if pd.isna(row['CR1']):
        # Remove the row where 'CR1' is empty
        df.drop(index, inplace=True)
        #print(df.isna().sum())

print(df.shape)

(3062, 53)
(2871, 53)


In [73]:
imputer = KNNImputer(n_neighbors=100)
df['AGEGRP'] = df['AGEGRP'].map({'infants': 1,'children': 2,'AYA': 3,'adults': 4,'seniors': 5,'elderly': 6 })

df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns, index=df.index)
df = df.round().astype(int)

In [ ]:
strat_element = 'AGEGRP'
df['split_col'] = df[strat_element] + df['CR1'] #for knn
size_test = 0.2 

X = df.loc[:, df.columns[(df.columns != 'CR1') & (df.columns != 'OSSTAT')]]
y = df['CR1']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=size_test, stratify=X['split_col'])
model = LogisticRegression()
X_train = X_train.drop([strat_element, 'split_col'], axis=1)#'TrialID', 
X_test = X_test.drop([ strat_element, 'split_col'], axis=1)#'TrialID',

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_prob = model.predict_proba(X_test)

#accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_prob[:,1])
mcc=matthews_corrcoef(y_test, y_pred)
print(roc_auc, mcc)

cm = confusion_matrix(y_test,y_pred)
print(cm)

explainer = shap.Explainer(model.predict,X_test)
shap_values = explainer.shap_values(X_test)

shap.summary_plot(shap_values, X_test, show=False, class_inds='original', plot_type="bar")

f = plt.gcf()
plt.xlim(0,0.15)
plt.legend(loc='lower right')
f.tight_layout()
f.savefig('',  facecolor='w', dpi=300, format="tiff")
plt.close()  

shap.plots.beeswarm(explainer(X_test), max_display=20, show=False, order=shap.Explanation.abs.mean(0))#

f = plt.gcf()
plt.xlim(-1,1)
f.tight_layout()
f.savefig('', facecolor='w', dpi=300, format="tiff")
plt.close()  

age_groups = {
    'Infants': X_test[X_test['AGE'] <= 2],
    'Children': X_test[(X_test['AGE'] > 2) & (X_test['AGE'] <= 15)],
    'AYA': X_test[(X_test['AGE'] > 15) & (X_test['AGE'] <= 40)],
    'Adults': X_test[(X_test['AGE'] > 40) & (X_test['AGE'] <= 65)],
    'Seniors': X_test[(X_test['AGE'] > 65) & (X_test['AGE'] <= 75)],
    'Elderly': X_test[X_test['AGE'] > 75]
}

for age_group_name, age_group_data in age_groups.items():
    X_age_group = age_group_data
    y_age_group = y_test[age_group_data.index]

    y_pred = model.predict(X_age_group)
    y_pred_prob_test =model.predict_proba(X_age_group)
    roc_auc_test = roc_auc_score(y_age_group, y_pred_prob_test[:,1])

    print(f" For {age_group_name},"f" AUC: {roc_auc_test:.4f}")
    cm = confusion_matrix(y_age_group,y_pred)
    print(cm)
    print(f"\nShap values for {age_group_name}:")

    explainer = shap.Explainer(model.predict,X_test)
    shap_values = explainer.shap_values(X_age_group) 
    
    shap.summary_plot(shap_values, X_age_group, show=False, class_inds='original', plot_type="bar")
    
    f = plt.gcf()
    plt.xlim(0,0.3)
    plt.legend(loc='lower right')
    f.tight_layout()
    f.savefig('' + age_group_name + '_bar.tiff', facecolor='w', dpi=300, format="tiff")
    plt.close()  
    
    shap.plots.beeswarm(explainer(X_age_group), max_display=20, show=False, order=shap.Explanation.abs.mean(0)) #lr, svm
    f = plt.gcf()
    plt.xlim(-1,1)
    f.tight_layout()
    f.savefig('' + age_group_name + '_beeswarm.tiff', facecolor='w', dpi=300, format="tiff")
    plt.close()  